In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/My Drive/Task1_2inputs")
!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm]"
import monai
from monai.data import  DataLoader, ImageDataset
from monai.transforms import (
    Resize, NormalizeIntensity, Activations, Compose, EnsureType, CenterSpatialCrop,
)

from monai.networks.nets import DenseNet121
from skimage.morphology import disk, binary_dilation, binary_erosion, remove_small_objects
import pandas as pd
import numpy as np
import nibabel as nib
import torch
import scipy.ndimage as nd
import os
import matplotlib.pyplot as plt
import matplotlib
import nibabel.processing
from monai.utils import set_determinism
from monai.data import decollate_batch
from torch.utils.tensorboard import SummaryWriter

from monai.transforms import (
    Compose, Activations,
    RandGaussianSmooth,
    EnsureType, RandAffine, RandRotate, Resize,  RandZoom, NormalizeIntensity,
    CenterSpatialCrop, LoadImage,
)
from os.path import exists

from sklearn.model_selection import StratifiedKFold, KFold
import os
set_determinism(seed=123)
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def generate(filenames_brains_segs, labels, output_brains_segs_path, names):
    number_runs = 8
    for i in range(1, number_runs):
        for j in range(0, len(names)):
            filepath = filenames_brains_segs[j]
            label = labels[j]
            if labels[j] == 0:
                if i < 5:
                    continue
            if exists(os.path.join(output_brains_segs_path + str(label),
                                  str(names[j]) + "_" + str(i) + "_brain_seg.nii.gz")) == True:
                continue
            loader = LoadImage(dtype=np.float32)
            image, metadata = loader(filepath)
            zoomImage = RandZoom(prob=1, min_zoom=0.7, max_zoom=1.3)
            rotateImage = RandRotate(prob=1, range_x=(0.4, 0.4))
            gaussImage = RandGaussianSmooth(prob=1, sigma_x=(0.25, 1.5))
            affineImage = RandAffine(prob=1, shear_range=(0.2, 0.2), mode="bilinear", padding_mode="zeros")
            resizeImage = Resize([192, 192, 64])
            cropImage = CenterSpatialCrop(roi_size=(128, 128, 64))
            nomalizeImage = NormalizeIntensity()
            if i != number_runs:
                image = zoomImage(image)
                image = rotateImage(image)
                image = gaussImage(image)
                image = affineImage(image)
            image = resizeImage(image)
            image = cropImage(image)
            image = nomalizeImage(image)

            volume = np.array(image, dtype=np.float32)
            # Convert the numpy array into nifti file
            volume = nib.Nifti1Image(volume, np.eye(4))
            nib.save(volume,
                     os.path.join(output_brains_segs_path + str(label),
                                  str(names[j]) + "_" + str(i) + "_brain_seg.nii.gz"))
        print(f'step {i} done')

def generateVal(filenames_brains_segs, labels, output_brains_segs_path, names):
    for j in range(0, len(names)):
        filepath = filenames_brains_segs[j]
        label = labels[j]
        if exists(os.path.join(output_brains_segs_path, str(names[j]) + "_brain_seg.nii.gz")) == True:
            continue
        loader = LoadImage(dtype=np.float32)
        image, metadata = loader(filepath)
        resizeImage = Resize([192, 192, 64])
        cropImage = CenterSpatialCrop(roi_size=(128, 128, 64))
        nomalizeImage = NormalizeIntensity()
        image = resizeImage(image)
        image = cropImage(image)
        image = nomalizeImage(image)

        volume = np.array(image, dtype=np.float32)
        # Convert the numpy array into nifti file
        volume = nib.Nifti1Image(volume, np.eye(4))
        nib.save(volume, os.path.join(output_brains_segs_path, str(names[j]) + "_brain_seg.nii.gz"))
if 1:
        root_dir = "data"
        data_dir = os.path.join(root_dir, "ATACH-2")
        raw_data = pd.read_csv(os.path.join(data_dir, "Labels_ATACH2_update.csv"))
        raw_data = raw_data.sort_values(by=["filename"], ascending=False)
        filenames_brains_segs = []
        labels = []
        names = []
        output_brains_segs_path = 'data/generated_brains_segs_'
        output_brains_segs_path_validation = 'data/validation'
        for _, c_row in raw_data.iterrows():
            filenames_brains_segs.append(
                os.path.join(data_dir, "ATACH2_original", str(c_row['filename']) + "_brain_seg.nii.gz"))
            labels.append(c_row['label'])
            names.append(c_row['filename'])
        labels = np.asarray(labels).astype(int)
        filenames_brains_segs = np.asarray(filenames_brains_segs)
        generate(filenames_brains_segs, labels, output_brains_segs_path, names)
        generateVal(filenames_brains_segs, labels, output_brains_segs_path_validation, names)


In [2]:
    root_dir = "data"
    data_dir = os.path.join(root_dir, "ATACH-2")
    if 1:
        raw_data = pd.read_csv(os.path.join(data_dir, "Labels_ATACH2.csv"))
        filenames = []
        labels = []
        filenamesHead = []
        names = []
        for _, c_row in raw_data.iterrows():
            filenames.append(os.path.join(data_dir, "ATACH2_original", str(c_row['filename']) + "_brain_seg.nii.gz"))
            labels.append(c_row['label'])
            names.append(c_row['filename'])
        labels = np.asarray(labels).astype(int)
        filenames = np.asarray(filenames)
        names = np.asarray(names)
        skf = StratifiedKFold(n_splits=3, shuffle=False)
        fullAUC = 0
        for index, (trainval_indices, test_indices) in enumerate(skf.split(filenames, labels)):
            print("Training +++ Valdation on fold " + str(index + 1))
            # Generate batches from indices
            ytrain, yval = labels[trainval_indices], labels[test_indices]
            xtrain, xval = filenames[trainval_indices], filenames[test_indices]
            nametrain, nameval = names[trainval_indices], names[test_indices]
            index_validation = -1
            metric_validation = -1
            print("training :")
            print(ytrain.shape)
            print(np.count_nonzero(ytrain))
            print("testing :")
            print(yval.shape)
            print(np.count_nonzero(yval))
            if 1:
                copyLabel = ytrain.copy()
                xtrain = []
                ytrain = []
                output_path = 'data/generated_brains_segs_'
                tmp = len(copyLabel)
                for aug in range(1, 8):
                    for j in range(0, tmp):
                        if copyLabel[j] == 0:
                            if aug < 5:
                                continue
                        xtrain = np.append(xtrain,
                                           os.path.join(output_path + str(copyLabel[j]),
                                                        str(nametrain[j]) + "_" + str(aug) + "_brain_seg.nii.gz"))
                        ytrain = np.append(ytrain, copyLabel[j])

                copyLabel = yval.copy()
                xval = []
                yval = []
                output_path = 'data/validation'
                tmp = len(copyLabel)
                for j in range(0, tmp):
                    xval = np.append(xval, os.path.join(output_path, str(nameval[j]) + "_brain_seg.nii.gz"))
                    yval = np.append(yval, copyLabel[j])
                class_sample_count = np.array([len(np.where(ytrain == t)[0]) for t in np.unique(ytrain)])
                print(class_sample_count)
                val_ds = ImageDataset(
                    image_files=xval, labels=yval)
                val_loader = DataLoader(val_ds, batch_size=10, shuffle=False,
                                        num_workers=10, pin_memory=torch.cuda.is_available())
                train_ds = ImageDataset(
                    image_files=xtrain, labels=ytrain)
                train_loader = DataLoader(train_ds, batch_size=10, shuffle=True,
                                          num_workers=10, pin_memory=torch.cuda.is_available())
                device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                model = monai.networks.nets.DenseNet121(
                    spatial_dims=3, in_channels=2, out_channels=1, dropout_prob=0.2).to(device)
                Loss = monai.losses.FocalLoss(reduction='mean', gamma=2, to_onehot_y=False)
                lr = 0.000001
                weight_decay = 1e-8
                optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
                model.cuda()
                val_interval = 1
                best_loss_metric = 200
                best_auc_metric = 0
                best_metric_epoch = -1
                epoch_loss_values = []
                epoch_loss_values_val = []
                auc_metric = monai.metrics.ROCAUCMetric()
                auc = []
                acc = []
                y_pred_trans = Compose([EnsureType(), Activations(sigmoid=True)])
                y_trans = Compose([EnsureType()])
                best_acc = -1
                metric_values = []
                writer = SummaryWriter()
                max_epochs = 1
                for epoch in range(max_epochs):
                    print("-" * 10)
                    print(f"epoch {epoch + 1}/{max_epochs}")
                    model.train()
                    epoch_loss = 0
                    step = 0
                    for batch_data in train_loader:
                        inputs, labels_tmp = batch_data[0].to(device), batch_data[1].to(device)
                        optimizer.zero_grad()
                        outputs = model(inputs)
                        epoch_len = len(train_ds) // train_loader.batch_size
                        loss = Loss(torch.sigmoid(outputs), torch.unsqueeze(labels_tmp, 1).float()).cuda()
                        step += 1
                        epoch_loss += loss.item()
                        loss.backward()
                        optimizer.step()
                    epoch_loss /= step
                    epoch_loss_values.append(epoch_loss)
                    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
                    if (epoch + 1) % val_interval == 0:
                        model.eval()
                        with torch.no_grad():
                            num_correct = 0.0
                            metric_count = 0
                            y_pred = torch.tensor([], dtype=torch.float32, device=device)
                            y = torch.tensor([], dtype=torch.long, device=device)
                            epoch_loss_val = 0
                            step_val = 0
                            for val_data in val_loader:
                                inputs, val_labels = val_data[0].to(
                                    device), val_data[1].to(device)
                                val_outputs = model(inputs)
                                y_pred1 = val_outputs.flatten()
                                y1 = val_labels
                                loss_val = Loss(torch.sigmoid(val_outputs),
                                                torch.unsqueeze(val_labels, 1).float()).cuda()
                                y_pred = torch.cat([y_pred, y_pred1], dim=0)
                                y = torch.cat([y, y1], dim=0)
                                step_val = step_val + 1
                                epoch_loss_val += loss_val.item()
                            y_onehot = [y_trans(i) for i in decollate_batch(y)]
                            y_pred_act = [y_pred_trans(i) for i in decollate_batch(y_pred)]
                            auc_metric(y_pred_act, y_onehot)
                            auc_result = auc_metric.aggregate()
                            auc_metric.reset()
                            auc.append(auc_result)
                            epoch_loss_val /= step_val
                            epoch_loss_values_val.append(epoch_loss_val)
                            del y_pred_act, y_onehot
                            if epoch_loss_val < best_loss_metric:
                                best_loss_metric = epoch_loss_val
                                best_metric_epoch = epoch + 1
                                torch.save(model.state_dict(),
                                           "best_metric_model_classification3d_" + str(index + 1) + ".pth")
                            if auc_result >= best_auc_metric:
                                best_auc_metric = auc_result
                                torch.save(model.state_dict(),
                                           "1_best_metric_model_classification3d_" + str(index + 1) + ".pth")
                            print(
                                "current epoch: {} current AUC: {:.4f}---- BEST AUC: {:.4f} ----current VAL LOSS: {:.4f} "
                                "Best VAL Loss: {:.4f} at epoch {}".format(
                                    epoch + 1, auc_result, best_auc_metric, epoch_loss_val, best_loss_metric,
                                    best_metric_epoch,
                                )
                            )

Training +++ Valdation on fold 1
training :
(202,)
57
testing :
(102,)
29
[435 399]
----------
epoch 1/1
epoch 1 average loss: 0.2274
current epoch: 1 current AUC: 0.4081---- BEST AUC: 0.4081 ----current VAL LOSS: 0.2718 Best VAL Loss: 0.2718 at epoch 1
Training +++ Valdation on fold 2
training :
(203,)
58
testing :
(101,)
28
[435 406]
----------
epoch 1/1
epoch 1 average loss: 0.2292
current epoch: 1 current AUC: 0.6042---- BEST AUC: 0.6042 ----current VAL LOSS: 0.2718 Best VAL Loss: 0.2718 at epoch 1
Training +++ Valdation on fold 3
training :
(203,)
57
testing :
(101,)
29
[438 399]
----------
epoch 1/1
epoch 1 average loss: 0.2396
current epoch: 1 current AUC: 0.5369---- BEST AUC: 0.5369 ----current VAL LOSS: 0.2930 Best VAL Loss: 0.2930 at epoch 1


In [3]:
!pip install pipreqs
!pipreqs 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
INFO: Successfully saved requirements file in /content/drive/MyDrive/Task1_2inputs/requirements.txt
